In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.patches as mpatches
import matplotlib.pyplot  as plt
import datetime
import copy
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score, f1_score
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Reshape
from keras.layers import Conv2D, Lambda, MaxPooling2D, Conv1D, MaxPooling1D, GlobalAveragePooling1D, Dropout
from keras.utils import np_utils
from keras import backend as K
import tensorflow as tf
import tqdm.notebook as tq
import gc
import psutil

psutil.virtual_memory()
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
import logging
tf.get_logger().setLevel(logging.ERROR)

In [3]:
num_classes = 2
epochs = 50

n_splits = 1

partition = 422*8*530
evaluation = 422*1*530
total = 2239250

In [4]:
def concat_two_df_and_drop_same_column_in_df2(DF1, DF2):
    display(DF1.head(1))
    display(DF2.head(1))
    DF2.drop(DF2.columns[DF2.columns.isin(DF1.columns)],axis=1,inplace=True)
#     DF1.drop(DF1.columns[DF1.columns.isin(DF2.columns)],axis=1,inplace=True)
    result = pd.concat([DF1, DF2], axis=1)
    display(result.head(1))
    return result
def count_list_val(arr):
    values, counts = np.unique(arr, return_counts=True)
    print(values, counts)
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [5]:
four_time = pd.read_csv("CSV/four_time.csv")
v_value = pd.read_csv("CSV/v_value.csv")
mic = pd.read_csv("CSV/mic.csv")
dwt_distance = pd.read_csv("CSV/sdtw_day_score_df.csv")
smart_meter = pd.read_csv("smart_meter.csv")
fdi = pd.read_csv("CSV/FDI.csv")

In [6]:
count_list_val(fdi['FDI'])

[0. 1.] [2127950  111300]


In [7]:
2127950/(2127950 + 111300)

0.9502958579881656

In [8]:
four_time.shape, v_value.shape, mic.shape, dwt_distance.shape, fdi.shape, smart_meter.shape

((2239250, 7),
 (2239250, 9),
 (2239250, 4),
 (2239250, 5),
 (2239250, 3),
 (107484000, 5))

In [9]:
result = concat_two_df_and_drop_same_column_in_df2(fdi, v_value)
result = concat_two_df_and_drop_same_column_in_df2(result, four_time)
result = concat_two_df_and_drop_same_column_in_df2(result, mic)
result = concat_two_df_and_drop_same_column_in_df2(result, dwt_distance)
result.columns

,MeterID,Day,FDI
0,1236,2009-07-15,1.0


,MeterID,Day,FDI,Tmax1,Tmax2,Tmin1,Cmax1,Cmax2,Cmin1
0,5212,2010-12-31,True,12,30,15,0.046464,0.047532,0.013561


,MeterID,Day,FDI,Tmax1,Tmax2,Tmin1,Cmax1,Cmax2,Cmin1
0,1236,2009-07-15,1.0,12,30,15,0.046464,0.047532,0.013561


,MeterID,Day,FDI,Tmax1,Tmax2,Tmin1,Cmax1,Cmax2,Cmin1
0,1236,2009-07-15,1.0,12,30,15,0.046464,0.047532,0.013561


,MeterID,Day,FDI,overnight,breakfast,daytime,evening
0,1236,2009-07-15,True,5.380015,15.5,24.5,38.0


...

,MeterID,Day,FDI,Tmax1,Tmax2,Tmin1,Cmax1,Cmax2,Cmin1,overnight,breakfast,daytime,evening,mic
0,1236,2009-07-15,1.0,12,30,15,0.046464,0.047532,0.013561,5.380015,15.5,24.5,38.0,0.624128


,MeterID,Day,FDI,Tmax1,Tmax2,Tmin1,Cmax1,Cmax2,Cmin1,overnight,breakfast,daytime,evening,mic
0,1236,2009-07-15,1.0,12,30,15,0.046464,0.047532,0.013561,5.380015,15.5,24.5,38.0,0.624128


,MeterID,Day,FDI,meter_dtw,meter_sdtw
0,1236,2009-07-15,True,2.283294,-68.276715


,MeterID,Day,FDI,Tmax1,Tmax2,Tmin1,Cmax1,Cmax2,Cmin1,overnight,breakfast,daytime,evening,mic,meter_dtw,meter_sdtw
0,1236,2009-07-15,1.0,12,30,15,0.046464,0.047532,0.013561,5.380015,15.5,24.5,38.0,0.624128,2.283294,-68.276715


Index(['MeterID', 'Day', 'FDI', 'Tmax1', 'Tmax2', 'Tmin1', 'Cmax1', 'Cmax2', 'Cmin1', 'overnight', 'breakfast', 'daytime', 'evening', 'mic', 'meter_dtw', 'meter_sdtw'], dtype='object')

In [10]:
elec = smart_meter['Electricity']
elec = np.array(elec).reshape(-1, 48)
elec = pd.DataFrame(elec)
print(elec.shape)
display(elec.head(3))

(2239250, 48)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
0,0.13407,0.129562,0.113616,0.18602,0.075942,0.133117,0.062728,0.064174,0.143932,0.142687,0.19219,0.141395,0.205161,0.060611,0.068435,0.059876,0.08228,0.13876,0.155309,0.060117,0.202968,0.186499,0.099944,0.065131,0.079839,0.073025,0.072445,0.135158,0.09189,0.109706,0.209875,0.161455,0.203446,0.075502,0.113514,0.140896,0.065063,0.053549,0.159253,0.154713,0.118059,0.13057,0.118797,0.056694,0.106094,0.08098,0.068501,0.204674
1,0.48300,0.062000,0.035000,0.06800,0.026000,0.013000,0.063000,0.063000,0.013000,0.013000,0.06900,0.047000,0.013000,0.027000,0.266000,0.420000,0.24200,0.22200,0.301000,0.062000,0.656000,0.151000,0.068000,0.031000,0.012000,0.149000,0.272000,0.416000,0.02300,0.384000,0.075000,0.036000,0.074000,0.194000,0.706000,0.148000,0.094000,0.156000,0.138000,0.099000,0.118000,0.16400,0.218000,0.412000,0.253000,0.61000,0.582000,0.292000
2,0.11400,0.041000,0.012000,0.04000,0.067000,0.017000,0.012000,0.064000,0.058000,0.012000,0.01700,0.069000,0.046000,0.012000,0.028000,0.067000,0.59300,0.10600,0.925000,0.185000,0.220000,0.200000,0.047000,0.258000,0.020000,0.059000,0.337000,0.261000,0.31300,0.238000,0.065000,0.022000,0.564000,0.582000,1.141313,1.387000,0.246000,0.346000,0.110000,0.104000,0.164000,0.40100,0.104000,0.138000,0.206000,0.36300,0.526000,0.326000


In [11]:
result_all= concat_two_df_and_drop_same_column_in_df2(result, elec)
result_all.columns = result_all.columns.astype(str)

,MeterID,Day,FDI,Tmax1,Tmax2,Tmin1,Cmax1,Cmax2,Cmin1,overnight,breakfast,daytime,evening,mic,meter_dtw,meter_sdtw
0,1236,2009-07-15,1.0,12,30,15,0.046464,0.047532,0.013561,5.380015,15.5,24.5,38.0,0.624128,2.283294,-68.276715


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
0,0.13407,0.129562,0.113616,0.18602,0.075942,0.133117,0.062728,0.064174,0.143932,0.142687,0.19219,0.141395,0.205161,0.060611,0.068435,0.059876,0.08228,0.13876,0.155309,0.060117,0.202968,0.186499,0.099944,0.065131,0.079839,0.073025,0.072445,0.135158,0.09189,0.109706,0.209875,0.161455,0.203446,0.075502,0.113514,0.140896,0.065063,0.053549,0.159253,0.154713,0.118059,0.13057,0.118797,0.056694,0.106094,0.08098,0.068501,0.204674


,MeterID,Day,FDI,Tmax1,Tmax2,Tmin1,Cmax1,Cmax2,Cmin1,overnight,breakfast,daytime,evening,mic,meter_dtw,meter_sdtw,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
0,1236,2009-07-15,1.0,12,30,15,0.046464,0.047532,0.013561,5.380015,15.5,24.5,38.0,0.624128,2.283294,-68.276715,0.13407,0.129562,0.113616,0.18602,0.075942,0.133117,0.062728,0.064174,0.143932,0.142687,0.19219,0.141395,0.205161,0.060611,0.068435,0.059876,0.08228,0.13876,0.155309,0.060117,0.202968,0.186499,0.099944,0.065131,0.079839,0.073025,0.072445,0.135158,0.09189,0.109706,0.209875,0.161455,0.203446,0.075502,0.113514,0.140896,0.065063,0.053549,0.159253,0.154713,0.118059,0.13057,0.118797,0.056694,0.106094,0.08098,0.068501,0.204674


In [12]:
df = result
X = df[['overnight', 'breakfast', 'daytime', 'evening',
       'Tmax1', 'Tmax2', 'Tmin1', 'Cmax1', 'Cmax2', 'Cmin1', 'mic',
       'meter_dtw', 'meter_sdtw']]
Y = df[["FDI"]]

In [13]:
def train_deep_model(df, x_col, y_col, score = False, split_num = 3, return_model=False):
    try:
        del model
    except:
        pass
    
    X = df[x_col]
    Y = df[y_col]
    
    x_train = X[:total]
    y_train = Y[:total]
    
    round = 1
    average_score = {}
    average_score["precision_score"] = 0
    average_score["recall_score"] = 0
    average_score["accuracy_score"] = 0
    average_score["roc_auc_score"] = 0
    average_score["f1_score"] = 0
    
    model = Sequential()

    model.add(Dense(12, input_dim=13, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.add(Dense(num_classes, activation='softmax'))

    print(model.summary())

    ES_function=keras.callbacks.EarlyStopping(monitor='accuracy', patience=10, min_delta=0.001)
    
    sss = StratifiedShuffleSplit(n_splits=split_num, test_size=0.3, random_state=1)
    for train_index, test_index in sss.split(x_train, y_train):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        Y_train, Y_test = np.array(Y.iloc[train_index]).reshape(-1) , np.array(Y.iloc[test_index]).reshape(-1)
    
        Y_train = to_categorical(Y_train)
        Y_test = to_categorical(Y_test)
        
        model.compile(loss='BinaryCrossentropy',
                    optimizer='adam', metrics=['accuracy'])
        history = model.fit(X_train, Y_train, epochs=epochs,  verbose=1, callbacks=[ES_function], class_weight = 
                        {0: 1, 1: 8}, validation_split=0.2,)
        
        y_pred = model.predict(X_test) # 預測值
        y_hat = np.argmax(y_pred, axis=1) # 
        y_test = np.argmax(Y_test, 1) # 
        
        average_score["precision_score"] += precision_score(y_test, y_hat)
        average_score["recall_score"] += recall_score(y_test, y_hat)
        average_score["accuracy_score"] += accuracy_score(y_test, y_hat)
        average_score["roc_auc_score"] += roc_auc_score(y_test, y_hat) 
        average_score["f1_score"] += f1_score(y_test, y_hat)

        if score:
            print("round: " + str(round))
            print("precision score: " + str(precision_score(y_test, y_hat)))
            print("recall_score: " + str(recall_score(y_test, y_hat)))
            print("accuracy_score: " + str(accuracy_score(y_test, y_hat)))
            print("roc_auc_score: " + str(roc_auc_score(y_test, y_hat)))
            print("f1_score: " + str(f1_score(y_test, y_hat)))
            print()

        round += 1
    for key, score in average_score.items():
        average_score[key] = score / split_num
        print("average_" + key + ": " + str(average_score[key]))
    if return_model==True:
        return model, history, average_score
    return average_score
    

In [14]:
num_classes = 2
epochs = 50

n_splits = 1

partition = 422*8*530
evaluation = 422*1*530
total = 2239250

In [23]:
try:
    del model
except:
    pass

round = 1
average_score = {}
average_score["precision_score"] = 0
average_score["recall_score"] = 0
average_score["accuracy_score"] = 0
average_score["roc_auc_score"] = 0
average_score["f1_score"] = 0

model = Sequential()

model.add(Dense(13, input_dim=61, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.add(Dense(num_classes, activation='softmax'))

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 13)                806       
_________________________________________________________________
dense_9 (Dense)              (None, 12)                168       
_________________________________________________________________
dense_10 (Dense)             (None, 12)                156       
_________________________________________________________________
dropout_2 (Dropout)          (None, 12)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 8)                 104       
_________________________________________________________________
dense_12 (Dense)             (None, 8)                 72        
_________________________________________________________________
dropout_3 (Dropout)          (None, 8)                

## 48 + 13 features

In [19]:
TITLE = "NN: 13 + 48 features"
x_col = result_all.drop(['Day', 'MeterID', 'FDI'], axis=1).columns
y_col = ['FDI']

X = result_all[x_col]
Y = result_all[y_col]

x_train = X[:total]
y_train = Y[:total]

split_num = 3

In [24]:
ES_function=keras.callbacks.EarlyStopping(monitor='recall', patience=3, min_delta=0.001)

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.4, random_state=1)
for train_index, test_index in sss.split(x_train, y_train):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    Y_train, Y_test = np.array(Y.iloc[train_index]).reshape(-1) , np.array(Y.iloc[test_index]).reshape(-1)

    Y_train = np_utils.to_categorical(Y_train)
    Y_test = np_utils.to_categorical(Y_test)
    
    model.compile(loss='categorical_crossentropy',
                optimizer='Adagrad', metrics=[tf.keras.metrics.Recall(
    thresholds=None, top_k=None, class_id=None, name=None, dtype=None
)])
    record = model.fit(X_train, Y_train, epochs=20, batch_size=128,  verbose=1, callbacks=[ES_function], class_weight = 
                    {0: 1, 1:10}, validation_split=0.2,)

Epoch 1/20
8398/8398 [==============================] - 17s 2ms/step - loss: 1.0590 - recall_1: 0.4447 - val_loss: 0.5304 - val_recall_1: 0.9504
Epoch 2/20
8398/8398 [==============================] - 15s 2ms/step - loss: 0.9521 - recall_1: 0.8125 - val_loss: 0.4664 - val_recall_1: 0.9504
Epoch 3/20
8398/8398 [==============================] - 15s 2ms/step - loss: 0.9404 - recall_1: 0.9212 - val_loss: 0.4445 - val_recall_1: 0.9504
Epoch 4/20
8398/8398 [==============================] - 15s 2ms/step - loss: 0.9301 - recall_1: 0.9443 - val_loss: 0.4390 - val_recall_1: 0.9504
Epoch 5/20
8398/8398 [==============================] - 16s 2ms/step - loss: 0.9303 - recall_1: 0.9472 - val_loss: 0.4396 - val_recall_1: 0.9504
Epoch 6/20
8398/8398 [==============================] - 16s 2ms/step - loss: 0.9260 - recall_1: 0.9486 - val_loss: 0.4442 - val_recall_1: 0.9504
Epoch 7/20
8398/8398 [==============================] - 15s 2ms/step - loss: 0.9237 - recall_1: 0.9487 - val_loss: 0.4459 - val_re

In [25]:
y_pred = model.predict(X_test, verbose=1, batch_size=128) # 預測值
y_hat = np.argmax(y_pred, axis=1) # 
y_test = np.argmax(Y_test, 1) # 

6998/6998 [==============================] - 6s 812us/step


In [26]:
average_score["recall_score"] += recall_score(y_test, y_hat.tolist())
average_score["accuracy_score"] += accuracy_score(y_test, y_hat)
average_score["roc_auc_score"] += roc_auc_score(y_test, y_hat) 
average_score["f1_score"] += f1_score(y_test, y_hat)
average_score["precision_score"] += precision_score(y_test, y_hat)


print("round: " + str(round))
print("precision score: " + str(precision_score(y_test, y_hat)))
print("recall_score: " + str(recall_score(y_test, y_hat)))
print("accuracy_score: " + str(accuracy_score(y_test, y_hat)))
print("roc_auc_score: " + str(roc_auc_score(y_test, y_hat)))
print("f1_score: " + str(f1_score(y_test, y_hat)))
print()

round += 1

round: 1
precision score: 0.20170329831133482
recall_score: 0.15480682839173404
accuracy_score: 0.9275371218041755
roc_auc_score: 0.5613803638422403
f1_score: 0.17517060834424125



## 48 features

In [15]:
TITLE = "NN: 13 + 48 features"
x_col = elec.columns.astype(str)
y_col = ['FDI']

X = result_all[x_col]
Y = result_all[y_col]

x_train = X[:total]
y_train = Y[:total]

split_num = 3

In [20]:
try:
    del model
except:
    pass

round = 1
average_score = {}
average_score["precision_score"] = 0
average_score["recall_score"] = 0
average_score["accuracy_score"] = 0
average_score["roc_auc_score"] = 0
average_score["f1_score"] = 0

model = Sequential()

model.add(Dense(13, input_dim=48, activation='relu'))
model.add(Dense(13, activation='relu'))
model.add(Dense(13, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(12, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.add(Dense(num_classes, activation='softmax'))

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 13)                637       
_________________________________________________________________
dense_9 (Dense)              (None, 13)                182       
_________________________________________________________________
dense_10 (Dense)             (None, 13)                182       
_________________________________________________________________
dropout_2 (Dropout)          (None, 13)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 12)                168       
_________________________________________________________________
dense_12 (Dense)             (None, 12)                156       
_________________________________________________________________
dropout_3 (Dropout)          (None, 12)               

In [21]:
ES_function=keras.callbacks.EarlyStopping(monitor='recall', patience=5, min_delta=0.001)

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.4, random_state=1)
for train_index, test_index in sss.split(x_train, y_train):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    Y_train, Y_test = np.array(Y.iloc[train_index]).reshape(-1) , np.array(Y.iloc[test_index]).reshape(-1)

    Y_train = np_utils.to_categorical(Y_train)
    Y_test = np_utils.to_categorical(Y_test)
    
    model.compile(loss='categorical_crossentropy',
                optimizer='Adagrad', metrics=[tf.keras.metrics.Recall(
    thresholds=None, top_k=None, class_id=None, name=None, dtype=None
)])
    record = model.fit(X_train, Y_train, epochs=20, batch_size=128,  verbose=1, callbacks=[ES_function], class_weight = 
                    {0: 1, 1:10}, validation_split=0.2,)

Epoch 1/20
8398/8398 [==============================] - 35s 4ms/step - loss: 1.0876 - recall_1: 0.0627 - val_loss: 0.6729 - val_recall_1: 0.9504
Epoch 2/20
8398/8398 [==============================] - 32s 4ms/step - loss: 0.9836 - recall_1: 0.9502 - val_loss: 0.5956 - val_recall_1: 0.9504
Epoch 3/20
8398/8398 [==============================] - 33s 4ms/step - loss: 0.9558 - recall_1: 0.9504 - val_loss: 0.5532 - val_recall_1: 0.9504
Epoch 4/20
8398/8398 [==============================] - 34s 4ms/step - loss: 0.9447 - recall_1: 0.9503 - val_loss: 0.5265 - val_recall_1: 0.9504
Epoch 5/20
8398/8398 [==============================] - 35s 4ms/step - loss: 0.9412 - recall_1: 0.9500 - val_loss: 0.5083 - val_recall_1: 0.9504
Epoch 6/20
8398/8398 [==============================] - 37s 4ms/step - loss: 0.9329 - recall_1: 0.9506 - val_loss: 0.4955 - val_recall_1: 0.9504
Epoch 7/20
8398/8398 [==============================] - 39s 5ms/step - loss: 0.9333 - recall_1: 0.9504 - val_loss: 0.4861 - val_re

In [18]:
y_pred = model.predict(X_test, verbose=1, batch_size=128) # 預測值
y_hat = np.argmax(y_pred, axis=1) # 
y_test = np.argmax(Y_test, 1) # 

6998/6998 [==============================] - 14s 2ms/step


In [19]:
average_score["recall_score"] += recall_score(y_test, y_hat.tolist())
average_score["accuracy_score"] += accuracy_score(y_test, y_hat)
average_score["roc_auc_score"] += roc_auc_score(y_test, y_hat) 
average_score["f1_score"] += f1_score(y_test, y_hat)
average_score["precision_score"] += precision_score(y_test, y_hat)


print("round: " + str(round))
print("precision score: " + str(precision_score(y_test, y_hat)))
print("recall_score: " + str(recall_score(y_test, y_hat)))
print("accuracy_score: " + str(accuracy_score(y_test, y_hat)))
print("roc_auc_score: " + str(roc_auc_score(y_test, y_hat)))
print("f1_score: " + str(f1_score(y_test, y_hat)))
print()

round += 1

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


round: 1


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


precision score: 0.0
recall_score: 0.0
accuracy_score: 0.9502958579881656
roc_auc_score: 0.5
f1_score: 0.0



In [ ]:
TITLE = "CNN: 13 features"
col = ['overnight', 'breakfast', 'daytime', 'evening',
       'Tmax1', 'Tmax2', 'Tmin1', 'Cmax1', 'Cmax2', 'Cmin1', 'mic',
       'meter_dtw', 'meter_sdtw']
start_time = timer(None)
model, history, score = train_deep_model(result, col, ['FDI'], return_model=True)
timer(start_time)